In [1]:
import pymongo
import pandas as pd
import time
import datetime
import xlwings as xw
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數
client = pymongo.MongoClient()
db = client['Fields']

In [2]:
# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)
path = r'C:\Users\ChastLai\Documents\Data\特別股股本.xlsx'
#path = r'C:\Users\ChastLai\Documents\Data\選擇權oi.xlsx'
#wb1 = xw.Book(path)
#func_all = wb1.macro("CM全部更新")
#func_all()
#wb1.save()
#wb1.close()
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('D processing.. ', k)
    D = v.drop('股票名稱', axis=1).set_index('股票代號')
    #D = v.drop('名稱', axis=1).set_index('代號')
    D.columns = [s[:8] for s in D.columns]
    D.columns = pd.to_datetime(D.columns)
    D = D.T.dropna(how='all', axis=0)
    D.columns = D.columns.astype(str)
    D['日期'] = D.index
    update_mongo(D, k)


D processing..  週RSI(5)
D processing..  週D(9)
D processing..  週K(9)
D processing..  ADX(14)
D processing..  -DI(14)
D processing..  +DI(14)
D processing..  W%R(10)
D processing..  W%R(5)
D processing..  DIF-MACD
D processing..  MACD
D processing..  DIF
D processing..  RSI(10)
D processing..  RSI(5)
D processing..  D(9)
D processing..  K(9)
D processing..  週MACD
D processing..  週DIF
D processing..  週RSI(10)


In [11]:


# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)
path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\月資料.xlsx'
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('M processing.. ', k)
    M = v.drop('股票名稱', axis=1).set_index('股票代號')
    M.columns = [s[:6]+'25' for s in M.columns]
    M.columns = pd.to_datetime(M.columns)
    M = M.T.dropna(how='all', axis=0)
    M.columns = M.columns.astype(str)
    M['日期'] = M.index
    update_mongo(M, k)

M processing..  全體及其關係人持股張數_M
M processing..  大股東及其關係人持股張數_M
M processing..  董監及其關係人持股張數_M


In [13]:
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)
path = r'C:\Users\ChastLai\Documents\Data\庫藏股票.xlsx'
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)

Q processing..  流動比率(%)_Q
Q processing..  稅後權益報酬率(%)_Q
Q processing..  稅後資產報酬率(%)_Q
Q processing..  庫藏股票(千)_Q


In [9]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\季資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)


path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\年資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Y processing.. ', k)
    Y = v.drop('股票名稱', axis=1).set_index('股票代號')
    Y.columns = [str(int(s[:4]) + 1) + '0331' for s in Y.columns]
    Y.columns = pd.to_datetime(Y.columns)
    Y = Y.T.dropna(how='all', axis=0)
    Y.columns = Y.columns.astype(str)
    Y['日期'] = Y.index
    update_mongo(Y, k)

path = r'C:\Users\ChastLai\Documents\Data\每日更新DB\日資料.xlsm'
wb1 = xw.Book(path)
func_all = wb1.macro("CM全部更新")
func_all()
wb1.save()
wb1.close()
exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('D processing.. ', k)
    D = v.drop('股票名稱', axis=1).set_index('股票代號')
    D.columns = [s[:8] for s in D.columns]
    D.columns = pd.to_datetime(D.columns)
    D = D.T.dropna(how='all', axis=0)
    D.columns = D.columns.astype(str)
    D['日期'] = D.index
    update_mongo(D, k)

# %%
db1 = client['Index']
#wb1.close()
#wb2.close()
wb3 = xw.Book(r"C:\Users\ChastLai\Documents\Data\每日更新DB\CMoney指數.xlsm")
func_ = wb3.macro("CM全部更新")
func_()
wb3.save()
wb3.close()
TWA00 = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='TWA00').iloc[:, 1:]
TWA00.rename({
    '開盤價': '還原開盤價', '最高價': '還原最高價', '最低價': '還原最低價', '收盤價': '還原收盤價',\
    '開盤價.1': '開盤價', '最高價.1': '最高價','最低價.1': '最低價', '收盤價.1': '收盤價' 
    }, axis=1, inplace=True)
VIX = pd.read_excel(r'C:\Users\ChastLai\Documents\Data\每日更新DB\Cmoney指數.xlsm', header=4, sheet_name='VIX').iloc[:, 1:]
for i in range(len(TWA00)):
    日期 = TWA00.iloc[i]['日期']
    db1['TWA00'].update_one({'日期': 日期}, update={'$set': TWA00.iloc[i].to_dict()}, upsert=True)
for i in range(len(VIX)):
    日期 = VIX.iloc[i]['日期']
    db1['VIX'].update_one({'日期': 日期}, update={'$set': VIX.iloc[i].to_dict()}, upsert=True)

In [2]:
# 季資料日期轉換
def change_Q(se: pd.Series):
    new = list()
    for s in se:
        if 'Q1' in s:
            s = s[:4] + '0515'
        elif 'Q2' in s:
            s = s[:4] + '0814'
        elif 'Q3' in s:
            s = s[:4] + '1114'
        elif 'Q4' in s:
            s = str(int(s[:4]) + 1) + '0331'
        new.append(s)
    return new

# 更新DB
def update_mongo(df: pd.DataFrame, dbname: str) -> None:
    for i, row in df.iterrows():
        db[dbname].update_one(filter={'日期': row['日期']}, update={'$set': row.to_dict()}, upsert=True)

path = r'C:\Users\ChastLai\Documents\Data\投資金額.xlsx'
# wb1 = xw.Book(path)
# func_all = wb1.macro("CM全部更新")
# func_all()
# wb1.save()
# wb1.close()

exl = pd.read_excel(path, sheet_name=None, header=4)
for k, v in exl.items():
    print('Q processing.. ', k)
    Q = v.drop('股票名稱', axis=1).set_index('股票代號')
    Q.columns = change_Q(Q.columns)
    Q.columns = pd.to_datetime(Q.columns)
    Q = Q.T.dropna(how='all', axis=0)
    Q.columns = Q.columns.astype(str)
    Q['日期'] = Q.index
    update_mongo(Q, k)

Q processing..  投資活動現金流量(千)_Q
